In [4]:
#미디어파이프를 이용해서 홍채영역 실시간 트래킹
import cv2
import mediapipe as mp

# MediaPipe 솔루션을 초기화합니다.
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 웹캠 입력을 받습니다.
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # 성능을 향상시키기 위해 이미지를 쓰지 않고 참조합니다.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # MediaPipe Face Mesh 처리를 합니다.
    results = face_mesh.process(image)

    # 이미지에 출력을 그리기 위해 다시 이미지를 BGR로 변환합니다.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 홍채 영역을 강조하기 위해 주요 랜드마크를 표시합니다.
            iris_landmarks = [
                face_landmarks.landmark[i] for i in range(468, 478)
            ]
            for landmark in iris_landmarks:
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                cv2.circle(image, (x, y), 2, (0, 0, 255), -1)

    cv2.imshow('MediaPipe Iris Tracking', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
# 사진 입력시 홍채영역 마킹
import cv2
import mediapipe as mp

# MediaPipe 솔루션을 초기화합니다.
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

def mark_iris(input_image_path, output_image_path):
    # 이미지를 읽어옵니다.
    image = cv2.imread(input_image_path)
    if image is None:
        raise ValueError("이미지를 로드할 수 없습니다.")

    # 이미지를 RGB로 변환합니다.
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # MediaPipe Face Mesh 처리를 합니다.
    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 홍채 영역을 찾기 위해 주요 랜드마크를 가져옵니다.
            iris_landmarks = [
                face_landmarks.landmark[i] for i in range(468, 478)
            ]

            # 홍채 영역의 좌표를 가져옵니다.
            for landmark in iris_landmarks:
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                cv2.circle(image, (x, y), 2, (0, 0, 255), -1)

    # 결과 이미지를 지정한 경로에 저장합니다.
    cv2.imwrite(output_image_path, image)

    return image

# 테스트할 이미지 경로를 지정합니다.
input_image_path = 'C:/Users/User/Desktop/programs/1234.jpg'
output_image_path = 'C:/Users/User/Desktop/programs/marked_iris_image.jpg'

# 함수 실행 및 결과 저장
marked_image = mark_iris(input_image_path, output_image_path)

# 결과 이미지를 화면에 표시합니다.
cv2.imshow('Marked Iris Image', marked_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [145]:
#홍채 가우시안 블러링
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 솔루션을 초기화합니다.
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

def blur_iris(input_image_path, output_image_path):
    # 이미지를 읽어옵니다.
    image = cv2.imread(input_image_path)
    if image is None:
        raise ValueError("이미지를 로드할 수 없습니다.")

    # 이미지를 RGB로 변환합니다.
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # MediaPipe Face Mesh 처리를 합니다.
    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 홍채 영역을 찾기 위해 주요 랜드마크를 가져옵니다.
            left_iris_landmarks = [
                face_landmarks.landmark[i] for i in range(468, 473)
            ]
            right_iris_landmarks = [
                face_landmarks.landmark[i] for i in range(473, 478)
            ]

            # 홍채 영역의 좌표를 가져옵니다.
            left_iris_points = np.array([[
                int(landmark.x * image.shape[1]),
                int(landmark.y * image.shape[0])
            ] for landmark in left_iris_landmarks])
            right_iris_points = np.array([[
                int(landmark.x * image.shape[1]),
                int(landmark.y * image.shape[0])
            ] for landmark in right_iris_landmarks])

            # 블러 처리를 위해 홍채 영역을 감싸는 컨벡스 헐(Convex Hull)을 계산합니다.
            left_hull = cv2.convexHull(left_iris_points)
            right_hull = cv2.convexHull(right_iris_points)

            # 블러 처리를 위한 마스크를 생성합니다.
            mask = np.zeros_like(image, dtype=np.uint8)
            cv2.fillConvexPoly(mask, left_hull, (255, 255, 255))
            cv2.fillConvexPoly(mask, right_hull, (255, 255, 255))

            # 원본 이미지를 블러 처리합니다.
            blurred_image = cv2.GaussianBlur(image, (51, 51), 13)

            # 마스크를 이용해 블러 처리된 홍채 영역을 원본 이미지에 합성합니다.
            mask_inv = cv2.bitwise_not(mask)
            image_masked = cv2.bitwise_and(image, mask_inv)
            blurred_iris = cv2.bitwise_and(blurred_image, mask)
            image = cv2.add(image_masked, blurred_iris)

    # 결과 이미지를 지정한 경로에 저장합니다.
    cv2.imwrite(output_image_path, image)

    return image

# 테스트할 이미지 경로를 지정합니다.
input_image_path = 'C:/Users/User/Desktop/programs/1234.jpg'
output_image_path = 'C:/Users/User/Desktop/programs/blurred_iris_image.jpg'

# 함수 실행 및 결과 저장
blurred_image = blur_iris(input_image_path, output_image_path)

# 결과 이미지를 화면에 표시합니다.
cv2.imshow('Blurred Iris Image', blurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
### 홍채영역 트래킹과 지문영역 블러링 (합침) (6/9수정)
import cv2
import mediapipe as mp
import math

# MediaPipe 솔루션을 초기화합니다.
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=10)

def calculate_distance(landmark1, landmark2, width, height):
    x1, y1 = int(landmark1.x * width), int(landmark1.y * height)
    x2, y2 = int(landmark2.x * width), int(landmark2.y * height)
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def calculate_finger_length(landmarks, width, height):
    # 손가락의 길이를 측정하기 위해 각 손가락의 관절 길이를 합산합니다.
    finger_lengths = []
    for finger_indices in [(4, 3, 2), (8, 7, 6), (12, 11, 10), (16, 15, 14), (20, 19, 18)]:
        finger_length = 0
        for i in range(len(finger_indices) - 1):
            finger_length += calculate_distance(landmarks[finger_indices[i]], landmarks[finger_indices[i+1]], width, height)
        finger_lengths.append(finger_length)
    return max(finger_lengths)

def blur_fingerprint_area(image, landmarks):
    h, w, _ = image.shape
    # 손가락 길이 계산
    finger_length = calculate_finger_length(landmarks, w, h)
    
    # 거리에 따라 블러 크기 조정
    blur_radius = int(max(15, min(50, finger_length // 4)))
    if blur_radius % 2 == 0:  # 홀수로 만들기
        blur_radius += 1
    ksize = (blur_radius, blur_radius)

    # 블러 처리 범위 조정
    blur_size = int(max(8, min(50, finger_length // 15)))
    
    for i in range(4, 21, 4):  # 각 손가락의 끝부분 (지문이 있는 부분) 인덱스
        x = int(landmarks[i].x * w)
        y = int(landmarks[i].y * h)
        # 지문 부분 주변의 사각형 영역 설정
        x_start, y_start = max(0, x-blur_size), max(0, y-blur_size)
        x_end, y_end = min(w, x+blur_size), min(h, y+blur_size)
        # 블러 처리할 영역이 유효한지 확인
        if x_start < x_end and y_start < y_end:
            image[y_start:y_end, x_start:x_end] = cv2.GaussianBlur(image[y_start:y_end, x_start:x_end], ksize, 10)
    return image

def blur_iris_area(image, iris_landmarks):
    h, w, _ = image.shape
    
    # 홍채 랜드마크의 최소 및 최대 x, y 좌표 찾기
    x_min = min(int(landmark.x * w) for landmark in iris_landmarks)
    y_min = min(int(landmark.y * h) for landmark in iris_landmarks)
    x_max = max(int(landmark.x * w) for landmark in iris_landmarks)
    y_max = max(int(landmark.y * h) for landmark in iris_landmarks)
    
    # 블러 크기 설정
    blur_radius = int(max(2, min(5, (x_max - x_min) // 42)))
    if blur_radius % 2 == 0:  # 홀수로 만들기
        blur_radius += 1
    ksize = (blur_radius, blur_radius)

    # 블러 처리할 영역 설정
    x_start, y_start = max(0, x_min - blur_radius), max(0, y_min - blur_radius)
    x_end, y_end = min(w, x_max + blur_radius), min(h, y_max + blur_radius)
    
    if x_start < x_end and y_start < y_end:
        image[y_start:y_end, x_start:x_end] = cv2.GaussianBlur(image[y_start:y_end, x_start:x_end], ksize, 15)
    
    return image

# 웹캠 입력을 받습니다.
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # 성능을 향상시키기 위해 이미지를 쓰지 않고 참조합니다.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # MediaPipe Face Mesh 처리를 합니다.
    face_results = face_mesh.process(image)

    # MediaPipe Hands 처리를 합니다.
    hand_results = hands.process(image)

    # 이미지에 출력을 그리기 위해 다시 이미지를 BGR로 변환합니다.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if face_results.multi_face_landmarks:
        for face_landmarks in face_results.multi_face_landmarks:
            # 홍채 영역을 강조하기 위해 주요 랜드마크를 표시합니다.
            iris_landmarks = [
                face_landmarks.landmark[i] for i in range(468, 478)
            ]
            image = blur_iris_area(image, iris_landmarks)

    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            image = blur_fingerprint_area(image, hand_landmarks.landmark)

    cv2.imshow('MediaPipe Iris and Hand Tracking', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()